<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson11_kmean_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #we will write our own k-means model from scratch and use it to cluster handwritten numbers from the MNIST dataset

In [2]:
#Unlike the lesson, I like to start with the main code body and then explore the 
#function definitions when I know what data they are being passed

#Data is a "bunch"
#A bunch is a dictionary but you address it like its an object
from sklearn.datasets import fetch_openml
data = fetch_openml(name='mnist_784')


In [ ]:
#skip this, just playing
print(data.keys())
print(data['DESCR'])
test = data['target']
print(test.shape)

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])
**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a resul

In [8]:
#I'm separating off the fetch_openml command because it takes so long so should be alone in cell

#So once again we have 70,000 observations
#bunch data has keys to hold lists of data and targets

#It does something I suspect inelegant here
#It splits 10% to test set (so 63,000 to train and 7000 to test)
#The acts on those vectors to put 33% of that into test data
#That gives test set of 2310 examples and 784 features
#and training set of 4690 examples

#The resultant training and test outputs are DataFrames and Series 
#(was this always the case? Or does bunch input lead to pandas output?)

import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.33)
len(X_train)

#There's an error in the Lesson code and this fixes it.
#Needs to be a numpy array or its can't be indexed later by [0]
#Otherwise it stays pandas
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [9]:
#Here's the key block of code that calls assign_data and kmeans

from scipy import stats
import math
from functools import reduce
import random

#for nlabeled in range(20,len(X_train),10):
nlabeled = 20
if True:
  print(nlabeled)
  ans = []
  k = 10 # 2 # 5 # 20
  if True:
    bestcenters, bestloss = kmeans(X_train, k)
    for rep in range(9):
      centers, loss = kmeans(X_train, k)
      if loss < bestloss: bestcenters, bestloss = centers, loss
    # How do we test the clustering that was discovered?
    # Assign testing points to clusters
    test_centerids, loss = assign_data(X_test, bestcenters)

    # Use the labeled examples to label the clusters
    train_centerids, loss = assign_data(X_train[:nlabeled], bestcenters)
    #print(train_centerids)
    #print(y_train[:nlabeled])
    labs = y_train[:nlabeled]

#    clust_labs = np.zeros(shape=(k))
    clust_labs = np.repeat(labs[0],k) #Like repmat https://numpy.org/doc/stable/reference/generated/numpy.repeat.html
    for i in range(k):
      mode = stats.mode(labs[train_centerids == i]).mode
      if len(mode) > 0: clust_labs[i] = mode[0]

# print(clust_labs)
    ans = ans + [(k,sum(clust_labs[test_centerids] == y_test)/len(y_test))]
#    plt.plot(X_test[clust_labs[test_centerids] == 0,0],X_test[clust_labs[test_centerids] == 0,1],'o',color='r')
#    plt.plot(X_test[clust_labs[test_centerids] == 1,0],X_test[clust_labs[test_centerids] == 1,1],'o',color='b')
#    plt.show()

#  print(ans)
  print(reduce((lambda x, y: x if x[1] > y[1] else y), ans)) #https://www.w3schools.com/python/python_lambda.asp

  #Display the final centerids and their losses
  labids, loss = assign_data(X_test, X_train[:nlabeled])
  print(nlabeled, sum(y_train[labids] == y_test)/len(y_test))

20
(10, 0.3878787878787879)
20 0.40735930735930737


In [20]:
#OK, no visualise the centroids as images

#We've seen this array_to_img before

#This lesson snippet produces an error
#!pip install keras=='2.3.1'
#from keras.preprocessing.image import array_to_img

#This fixes it
#!pip install keras==2.8
import keras.utils as image
#from keras.preprocessing.image import img_to_array

#Get the ids for the final result 
train_centerids, loss = assign_data(X_train, bestcenters)
test_centerids, loss = assign_data(X_test, bestcenters)

clust_labs = np.repeat(labs[0],k) #like repmat in matlab. I guess it tiles the labs so they can be put along the looped image outputs

#Go through the centers
for i in range(len(bestcenters)):

  #display a center (best)
  display(array_to_img(np.reshape(bestcenters[i],(28,28,1)), scale=False))

  #Assign to clust_labs the label for this centroid
  clust_labs[i] = y_train[train_centerids == i][0]
  print(clust_labs[i])
#  mode = stats.mode(y_train[train_centerids == i]).mode
#  print(mode[0])
#  if len(mode) > 0: clust_labs[i] = mode[0]

#classification accuracy
sum(clust_labs[test_centerids] == y_test)/len(y_test)

ImportError: ignored

In [4]:
#With these two functions, we can build a kmeans model, which takes in the data 
#and number of clusters, k and iteratively builds k clusters and updates them relative to the loss.

def kmeans(data, k):

  #num obs
  n = len(data)
  #num "cols"
  d = len(data[0])

  # grab the centers from random points
  centers = data[[random.randint(0,n-1) for i in range(k)]]

  #Now, starting with random centers, iterate, using assign_data to find loss for those centers
  #and new indices into centers where distances to points are minimised.
  #Then compute means to find the pixel values of the (mean) centers and try again
  #Terminate loop when loss equals 0 (no more improvement)
  oldloss = 0 #For comparison of current loss to 0. Weird name
  loss = 1    #Start off with dummy value of loss that isn't zero
  while oldloss != loss:
    oldloss = loss  #I guess this is guaranteed mathematcially toi find zero, which is why we don't wait until it falls below a near-zero value
    centerids, loss = assign_data(data,centers) #Find centerids that minimise distances and their loss
    centers = compute_means(data, centerids, k) #Find their pixel values

  return(centers, loss)

In [5]:
#an assign_data function, which takes the data and the centers for each cluster 
#and makes an assignment of each datapoint in data to the closest of the centers, centerids.

#So a "datapoint" must be an image and so dims = num pixels. A "center" must be the same, 
#they're all just points in multi-dimentional pixel space. centerid, though is an index into center
#locations that gives one with the smallest distance to datapoints

def assign_data(data,centers):
  
  #We extract n, the number of datapoints, d, the dimensionality of the datapoints, 
  #and k the number of centers.

  # n is the number of data points
  n = len(data)
  # d is the dimensionality of the data points
  d = len(data[0])
  # k is the number of clusters
  k = len(centers)

  #Next, we need to compute the squared distance between each center and each data point.

  # first, subtract the set of centers from each data point
  #Reshaping the data to be 1 x n x d, and the centers to be k x 1 x d signals to 
  #numpy that when it subtracts these two arrays, it creates an array of shape k x n x d. 
  #That is, it computes all combinations of the k centers and the n datapoints for each of the d dimensions. 
  #We assign those differences to res.
  res = np.reshape(data,(1,n,d))-np.reshape(centers,(k,1,d))
  
  # sum the squared differences
  #Squaring each of the differences, then summing along dimension 2 
  #--- that’s the components of the vectors --- produces the sum of squared distances, 
  #which is the squared distance between the centers and the datapoints. 
  #The resulting array is of shape k x n.
  res2 = np.add.reduce(res**2,2)


  # assign each data point to its closest center
  #We want to know which center has the smallest squared distance for each data point. 
  #argmin produces the index of an array with the smallest value along the given dimension.
  #Here, we’re using dimension 0, which varies over the k centers. 
  #centerids is now an array with one integer for each datapoint that indicates which of the centers is closest.
  centerids = np.apply_along_axis(np.argmin,0,res2)

  # While we're here, make a note of the loss
  loss = sum(np.apply_along_axis(np.min,0,res2))

  return(centerids, loss)

In [6]:
#Next we'll compute the mean of each of the k centers using the data and their 
#centerids assignments. compute_means takes the data and the center ids and 
#computes the centers by averaging all of the datapoints with the same id. 
#This will be used to update the centers.


def compute_means(data, centerids, k):

  # n is number of data points
  n = len(data)
  # d is dimensionality of the data points
  d = len(data[0])
 
 #I think this more precisely means to create an array of zeros
 #that will soon hold the new centers
  # Zero out the centers
  centers = np.zeros(shape=(k,d))
 
  # loop over the clusters
  for i in range(k):

    #Get the pixel values for all data examples (each is a j) that have a certain center if i
    # Gather the data points assigned to cluster i
    cols = np.array([data[j] for j in range(n) if centerids[j] == i])


    # Average to get mean for that cluster
    
    if len(cols) == 0: #I guess it can happen that ids are missing?
      centers[i] = data[random.randint(0,n-1)]
    else:
      centers[i] = cols.mean(0)

 #return the new means
  return(centers)